In [51]:
# import libs
import pyarrow.parquet as pq
import pyarrow as pa
import s3fs

In [42]:
# MinIO configuration
minio_endpoint = "http://minio.d4n-storage.svc.cluster.local:9000"  # Replace with your MinIO server URL
minio_access_key = "serviceaccount"                # Replace with your MinIO access key
minio_secret_key = "PassWordD4Now"                # Replace with your MinIO secret key

In [43]:
# Initialize S3 filesystem for MinIO
fs = s3fs.S3FileSystem(
    endpoint_url=minio_endpoint,
    key=minio_access_key,
    secret=minio_secret_key
)

In [ ]:
# path to the minio parquet data 
staging_parquet_path = "staging/Survey/data.parquet"

In [45]:
# Read Parquet file from MinIO
rga_table = pq.read_table(staging_parquet_path, filesystem=fs)

In [46]:
# Convert to Pandas DataFrame (optional)
df = rga_table.to_pandas()
df.head()

,country,sex,education,ID,age,marital_status,hh_size,child,employment_status,income_loss,...,age_cat2,education2,edu_age,edu_child,young_child,hh_comp_mar,countryweightf,countryweightm,_merge,weights_sample
0,Ethiopia,Men,Technical,None,26,Single,5,0,Own Account Worker,yes,...,25-34,Tertiary Education and More,0.0,0.0,0,Single without children,1,1,Matched (3),1
1,Ethiopia,Men,Tertiary,None,37,Married,6,1,Formal employee,yes,...,35-44,Tertiary Education and More,0.0,0.0,0,Partnered with children,1,1,Matched (3),0
2,Ethiopia,Men,Secondary,None,58,Single,1,0,Own Account Worker,No,...,45-59,Secondary Education or Less,0.0,0.0,0,Single without children,1,1,Matched (3),1
3,Ethiopia,Women,Secondary,None,26,Single,3,0,Own Account Worker,yes,...,25-34,Secondary Education or Less,0.0,0.0,0,Single without children,1,1,Matched (3),1
4,Ethiopia,Men,Technical,None,33,Married,3,0,Own Account Worker,yes,...,25-34,Tertiary Education and More,0.0,0.0,0,Partnered without children,1,1,Matched (3),0


In [47]:
import pygwalker as pyg

# Launch PyGWalker to explore the DataFrame
walker = pyg.walk(df, kernel_computation=True)

Box(children=(HTML(value='\n<div id="ifr-pyg-00062b4131812f8ep3dy9K0CinPx7JTI" style="height: auto">\n    <hea…

In [ ]:
# MinIO path to save the Parquet file
aggregated_parquet_path = "aggregated/Survey/data.parquet"

In [49]:
# Convert the Pandas DataFrame to a PyArrow Table
result_table = pa.Table.from_pandas(df)

In [50]:
# Save the PyArrow Table as a Parquet file to MinIO
pq.write_table(result_table, aggregated_parquet_path, filesystem=fs)